In [1]:
import pandas as pd
import numpy as np

In [2]:
import matplotlib.pyplot as plt

In [3]:
def refine(paramdf):
    df = paramdf.copy()
    df = df[df['Mins'] >= 450] # 450분 이상 출전한 자료만 
    df.drop(['OwnG'],axis=1,inplace = True)
    df.replace('-','0' , inplace=True)
    df.drop(['Apps'],axis=1,inplace=True)
    df = df[df['Position'] != 'GK'] # 골키퍼 삭제
    df = df[df['Position'] != 'Goalkeeper']
    df = df[df['Position'] != 'Substitute']
    df['Position'].replace('Forward','FW',inplace=True)
    df['Position'].replace('Midfielder','M(C)',inplace=True) # midfielder 는 M(C) 중앙 미드필더로 치환
    df['Position'].replace('Defender','D(C)',inplace=True)
    df.reset_index(inplace=True)
    df.drop(['index'],axis=1,inplace=True)
    return df

좌표로 나타낼 때 M(CenterLeftRight)이런 새끼들이 있어서 오른쪽 왼쪽 구분하는게 어려워서 측면과 중앙정도로만 나눔
- FW 하나만 있는애들은 0,4값 주고 fw랑 AM(공격형미필)같이 섞여있는애들은 보정해서 늠 
- 보정은 예를들어 FW,AM(CL) 일 때 0,4 + 1,3  -> 1,7에서 fw가 아닌 다른 포지션의 x값 평균을 보정해서 넘 -> 1,7 + 0.5,0 -> 1.5,7
- 이렇게 된 거를 포지션 개수로 나눔(중앙의 좌표를 얻는거) => (0.75,3.5) 이게 FW,AM(CL)의 포지션 좌표

In [4]:
def posConvert(paramdf):
    df = paramdf.copy()
    
    pos_dic = {'FW':[0,4], 'AM(L)':[2,3],'AM(CL)':[1,3],'AM(C)':[0,3],'AM(CR)':[1,3], 'AM(R)':[2,3],
            'M(L)':[2,2], 'M(CL)':[1,2],'M(C)':[0,2],'M(CR)':[1,2],'M(R)':[2,2], 'DMC':[0,1],
            'D(L)':[2,0], 'D(CL)':[1,0], 'D(C)':[0,0], 'D(CR)':[1,0],'D(R)':[2,0], 
           'AM(CLR)':[1,3],'M(CLR)':[1,2],'D(CLR)':[1,0],
            'AM(LR)' :[2,3],'M(LR)':[2,2], 'D(LR)':[2,0]}
    
    pos_xlist=[0*i for i in range(len(df['Position']))]
    pos_ylist=[0*i for i in range(len(df['Position']))]
    for i in range(len(df['Position'])):
        count = 0
        correction_x=False # FW는 측면 정보가 없어서 x좌표에 보정 늘거야, FW와 다른 포지션 같이 있는 데이터만
        for posdic in pos_dic.keys():
            if df['Position'][i].find(posdic) != -1:
                if posdic == 'FW':
                    correction_x = True
                pos_xlist[i] += pos_dic[posdic][0]
                pos_ylist[i] += pos_dic[posdic][1]
                count+=1
        if count == 0:
            print(df['Position'][i])

        if correction_x and count != 1:
            pos_xlist[i] += pos_xlist[i]/(count-1) # FW x값 보정, FW좌표를 뺀 나머지 포지션의 좌표 평균을 x에 더해줌
        pos_xlist[i] = pos_xlist[i]/count
        pos_ylist[i] = pos_ylist[i]/count
        
        position_df = pd.DataFrame(zip(pos_xlist,pos_ylist), columns=['posX','posY'])
    return position_df
    

In [5]:
year = '2021-2022'
epl = pd.read_csv('./dataset/refinedEPL'+year+'.csv',index_col=0)
laliga = pd.read_csv('./dataset/refinedLaLiga'+year+'.csv',index_col=0)
bundes = pd.read_csv('./dataset/refinedGermany-Bundesliga-'+year+'.csv',index_col=0)
serie = pd.read_csv('./dataset/refinedItaly-Serie-A-'+year+'.csv',index_col=0)
ligue = pd.read_csv('./dataset/refinedFrance-Ligue-1-'+year+'.csv',index_col=0)

In [6]:
# row 생략 없이 출력
pd.set_option('display.max_rows', 100)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

Mins: Minutes played   Assists: Total assists   KeyP: Key passes per game   AvgP: Passes per game   PS%: Pass success percentage
Crosses: Crosses per game   LongB: Long balls per game   ThrB: Through balls per game 
Goals: Total goals   Assists: Total assists   SpG: Shots per gameKeyP: Key passes per game   Drb: Dribbles per game   
Fouled: Fouled per game   Off: Offsides per game   Disp: Dispossessed per game   UnsTch: Bad control per game
Tackles: Tackles per game   Inter: Interceptions per game   Fouls: Fouls per game
Offsides: Offside won per game   Clear: Clearances per game   Drb: Dribbled past per game   Blocks: OutfielderBlockPerGame
OwnG: Own goals

In [7]:
total = pd.concat([epl,laliga,bundes,serie,ligue])
total

,Name,Club,Age,Position,Apps,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,OwnG,Rating,Goals,Assists,SpG,KeyP,Fouled,Off,Disp,UnsTch,AvgP,PS%,Crosses,LongB,ThrB
0,Liam McCarron,Leeds,21,Midfielder,0(1),9,-,-,-,-,-,-,-,-,5.91,-,-,-,-,-,-,-,-,2.0,100.0,-,-,-
1,Lewis Bate,Leeds,19,Midfielder,1(2),147,0.3,0.3,1,-,0.3,2.7,-,-,5.82,-,-,0.3,-,0.3,-,0.7,3,20.3,73.8,-,1,-
2,Freddie Woodman,Newcastle,25,GK,4,360,-,-,-,-,-,-,-,-,5.81,-,-,-,-,-,-,-,-,26.8,58.9,-,6,-
3,Sead Kolasinac,Arsenal,28,"D(CL),DMC,M(L)",1(1),91,-,2,1,-,0.5,-,-,-,5.72,-,-,-,-,-,-,-,-,7.0,92.9,-,-,-
4,Isaac Price,Everton,18,Midfielder,0(1),12,-,-,-,-,-,-,-,-,5.70,-,-,1,-,-,-,1,-,2.0,50.0,-,-,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,Gaëtan Laborde,Montpellier,28,"AM(CLR),FW",4,342,0.5,1.3,0.8,-,1.3,0.5,-,-,7.47,3,-,3.5,1.0,0.8,-,0.8,2.0,23.3,60.2,0.3,1.0,0.3
601,Benjamin Bourigeaud,Rennes,28,M(CLR),38,3215,1.2,0.8,0.7,-,1,1.1,0.1,-,7.44,11,12,1.3,3.2,0.2,0.1,0.5,1.1,42.1,76.9,2.7,2.7,0.1
602,Dimitri Payet,Marseille,35,"M(CLR),FW",28(3),2597,0.5,0.3,0.3,-,0.1,0.8,-,-,7.44,12,10,2.2,3.2,1.4,0.1,1.1,1.4,45.5,81.8,2.3,2.6,0.5
603,Martin Terrier,Rennes,25,"AM(CL),FW",37,2805,1.5,0.5,0.8,-,0.5,0.5,0.1,-,7.41,21,3,2.4,1.1,1.1,0.2,0.6,1.6,24.9,81.1,0.2,1.0,0.1


In [8]:
total.isna().sum()

Name        0
Club        0
Age         0
Position    0
Apps        0
Mins        0
Tackles     0
Inter       0
Fouls       0
Offsides    0
Clear       0
Drb         0
Blocks      0
OwnG        0
Rating      0
Goals       0
Assists     0
SpG         0
KeyP        0
Fouled      0
Off         0
Disp        0
UnsTch      0
AvgP        0
PS%         0
Crosses     0
LongB       0
ThrB        0
dtype: int64

In [9]:
refined = total.copy() # p 데이타는 

In [10]:
refined = refine(refined)
refined

,Name,Club,Age,Position,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,Rating,Goals,Assists,SpG,KeyP,Fouled,Off,Disp,UnsTch,AvgP,PS%,Crosses,LongB,ThrB
0,Fábio Silva,Wolves,19,FW,565,0.1,0,0.6,0,0,0.4,0,6.09,0,0,0.9,0.1,0.3,0.2,0.6,0.8,6.2,68.4,0,0.1,0
1,Salomón Rondón,Everton,32,FW,783,0.3,0,0.6,0,0.5,0.5,0,6.19,1,1,1.1,0.4,0.2,0.3,0.5,1.3,9.3,62.4,0.1,0.2,0
2,Moussa Djenepo,Southampton,23,AM(LR),455,0.8,0.2,0.8,0,0.4,0.7,0.1,6.18,0,0,0.9,0.5,0.8,0,0.8,1.9,11.7,75.7,0.2,0.1,0.1
3,Axel Tuanzebe,Aston Villa,24,D(CR),506,0.6,0.6,0.3,0.4,1.3,0.3,0.4,6.18,0,0,0.2,0,0.3,0,0,0.3,26.7,86.3,0,1.1,0
4,André Gomes,Everton,28,M(CLR),619,1.1,0.1,1.7,0,0.5,1.7,0.1,6.17,0,1,1,0.3,0.6,0,1.1,1.5,19.7,77.2,0.4,0.9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1876,Tetê,Lyon,22,AM(R),585,1.1,0.3,0.4,0,0.1,1.1,0,7.48,2,5,1.9,1.4,2.0,0.3,1.2,1.6,24.7,82.9,0.4,0.9,0.1
1877,Benjamin Bourigeaud,Rennes,28,M(CLR),3215,1.2,0.8,0.7,0,1,1.1,0.1,7.44,11,12,1.3,3.2,0.2,0.1,0.5,1.1,42.1,76.9,2.7,2.7,0.1
1878,Dimitri Payet,Marseille,35,"M(CLR),FW",2597,0.5,0.3,0.3,0,0.1,0.8,0,7.44,12,10,2.2,3.2,1.4,0.1,1.1,1.4,45.5,81.8,2.3,2.6,0.5
1879,Martin Terrier,Rennes,25,"AM(CL),FW",2805,1.5,0.5,0.8,0,0.5,0.5,0.1,7.41,21,3,2.4,1.1,1.1,0.2,0.6,1.6,24.9,81.1,0.2,1.0,0.1


In [11]:
posConvert(refined)

,posX,posY
0,0.0,4.0
1,0.0,4.0
2,2.0,2.5
3,1.0,0.0
4,1.0,2.0
...,...,...
1876,2.0,2.5
1877,1.0,2.0
1878,1.0,3.0
1879,1.0,3.0


In [12]:
refined = pd.concat([refined,posConvert(refined)],axis=1)

In [13]:
for column in refined.columns[:]:
    if column == 'Position':
        continue
    elif column == 'Club':
        continue
    elif column == 'Name':
        continue
    else:
        refined[column] = refined[column].astype(float)
refined

,Name,Club,Age,Position,Mins,Tackles,Inter,Fouls,Offsides,Clear,Drb,Blocks,Rating,Goals,Assists,SpG,KeyP,Fouled,Off,Disp,UnsTch,AvgP,PS%,Crosses,LongB,ThrB,posX,posY
0,Fábio Silva,Wolves,19.0,FW,565.0,0.1,0.0,0.6,0.0,0.0,0.4,0.0,6.09,0.0,0.0,0.9,0.1,0.3,0.2,0.6,0.8,6.2,68.4,0.0,0.1,0.0,0.0,4.0
1,Salomón Rondón,Everton,32.0,FW,783.0,0.3,0.0,0.6,0.0,0.5,0.5,0.0,6.19,1.0,1.0,1.1,0.4,0.2,0.3,0.5,1.3,9.3,62.4,0.1,0.2,0.0,0.0,4.0
2,Moussa Djenepo,Southampton,23.0,AM(LR),455.0,0.8,0.2,0.8,0.0,0.4,0.7,0.1,6.18,0.0,0.0,0.9,0.5,0.8,0.0,0.8,1.9,11.7,75.7,0.2,0.1,0.1,2.0,2.5
3,Axel Tuanzebe,Aston Villa,24.0,D(CR),506.0,0.6,0.6,0.3,0.4,1.3,0.3,0.4,6.18,0.0,0.0,0.2,0.0,0.3,0.0,0.0,0.3,26.7,86.3,0.0,1.1,0.0,1.0,0.0
4,André Gomes,Everton,28.0,M(CLR),619.0,1.1,0.1,1.7,0.0,0.5,1.7,0.1,6.17,0.0,1.0,1.0,0.3,0.6,0.0,1.1,1.5,19.7,77.2,0.4,0.9,0.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1876,Tetê,Lyon,22.0,AM(R),585.0,1.1,0.3,0.4,0.0,0.1,1.1,0.0,7.48,2.0,5.0,1.9,1.4,2.0,0.3,1.2,1.6,24.7,82.9,0.4,0.9,0.1,2.0,2.5
1877,Benjamin Bourigeaud,Rennes,28.0,M(CLR),3215.0,1.2,0.8,0.7,0.0,1.0,1.1,0.1,7.44,11.0,12.0,1.3,3.2,0.2,0.1,0.5,1.1,42.1,76.9,2.7,2.7,0.1,1.0,2.0
1878,Dimitri Payet,Marseille,35.0,"M(CLR),FW",2597.0,0.5,0.3,0.3,0.0,0.1,0.8,0.0,7.44,12.0,10.0,2.2,3.2,1.4,0.1,1.1,1.4,45.5,81.8,2.3,2.6,0.5,1.0,3.0
1879,Martin Terrier,Rennes,25.0,"AM(CL),FW",2805.0,1.5,0.5,0.8,0.0,0.5,0.5,0.1,7.41,21.0,3.0,2.4,1.1,1.1,0.2,0.6,1.6,24.9,81.1,0.2,1.0,0.1,1.0,3.0


In [15]:
refined.to_csv('./dataset/refinedDataSet_withPos_'+year+'.csv',encoding='utf-8-sig')